# **ETL - Registros**

In [1]:
# [Configuración] Importación de librerías
import pandas as pd
import numpy as np
import unicodedata
import re
import os

In [2]:
# [Función] Limpiar nombres de columnas
def limpiar_texto(texto):
    if not isinstance(texto, str):
        return texto
    texto = texto.replace("ã±", "ñ").replace("Ã±", "ñ").replace("ï¿½", "ñ")
    texto = unicodedata.normalize("NFKC", texto)
    texto = re.sub(r'\s+', '_', texto.strip().lower())
    texto = texto.replace("_", "")
    return texto

def read_file(ruta):
    archivos = sorted([f for f in os.listdir(ruta) if f.endswith(".csv")])
    dataframes = {}
    for archivo in archivos:
        path = os.path.join(ruta, archivo)
        df = pd.read_csv(path, encoding='latin-1', low_memory=False)

        # Limpieza de nombres de columnas
        columnas_limpias = [limpiar_texto(col) for col in df.columns]
        df.columns = columnas_limpias
        df.columns = df.columns.str.lower()
        df = df[sorted(df.columns)]
        dataframes[archivo] = df
    print(f"✅ {len(dataframes)} archivos cargados desde {ruta}")
    return dataframes

def check_cols(dataframes, unidad):
    print(f"\n======={unidad}======\n")
    columnas_por_archivo = {
        nombre: set(df.columns)
        for nombre, df in dataframes.items()
    }
    # Columnas comunes a todos los archivos
    comunes = set.intersection(*columnas_por_archivo.values())

    # Columnas únicas por archivo
    exclusivas = {
        nombre: columnas - comunes
        for nombre, columnas in columnas_por_archivo.items()
    }
    print("✅ Columnas comunes a todos los archivos:")
    print(sorted(comunes))

    print("\n❌ Columnas exclusivas por archivo:")
    for nombre, cols in exclusivas.items():
        print(f"📄 {nombre}: {sorted(cols)}")


In [3]:
# [Configuración] Lectura de los archivos CSV
ruta_atl = "../ARCHIVOS/RAW/raw_atlantico"
ruta_col = "../ARCHIVOS/RAW/raw_colombia"
dfs_atl = read_file(ruta_atl)
dfs_col = read_file(ruta_col)

✅ 6 archivos cargados desde ../ARCHIVOS/RAW/raw_atlantico
✅ 12 archivos cargados desde ../ARCHIVOS/RAW/raw_colombia


In [4]:
check_cols(dfs_col, "Colombia")
check_cols(dfs_atl, "Atlantico")


=======Colombia======

✅ Columnas comunes a todos los archivos:
['ajuste', 'ano', 'area', 'cbmte', 'cerdef', 'codase', 'coddpton', 'coddptoo', 'coddptor', 'codeve', 'codmunn', 'codmuno', 'codmunr', 'codpaiso', 'codpre', 'codsub', 'confin', 'confirmados', 'departamentoocurrencia', 'departamentoresidencia', 'edad', 'estadofinaldecaso', 'fecaju', 'feccon', 'fecdef', 'fechanto', 'fechos', 'fecnot', 'fmfuerza', 'fmgrado', 'fmunidad', 'gpcarcela', 'gpdesmovi', 'gpdesplaz', 'gpdiscapa', 'gpgestan', 'gpindigen', 'gpmadcom', 'gpmigrant', 'gpotros', 'gppobicfb', 'gppsiquia', 'gpvicvio', 'grupob', 'inisin', 'municipioocurrencia', 'municipioresidencia', 'nombreevento', 'nomestfcaso', 'ocupacion', 'pachos', 'peretn', 'semana', 'sexo', 'tipcas', 'tipss', 'unimed']

❌ Columnas exclusivas por archivo:
📄 Datos_2018_210.csv: ['codpaisr', 'consecutive', 'consecutiveorigen', 'departamentonotificacion', 'estrato', 'fecarcxl', 'fuente', 'municipionotificacion', 'nacionalidad', 'nombrenacionalidad', 'nomgru

In [5]:
# Descartar variables poco informativas
def delete(dataframes, unidad):
    print(f"\n======={unidad}======\n")
    delete = [
        'ajuste', 'barver', 'cbmte', 'cenpobla', 'cerdef', 'codase', 'coddptor', 'codpaiso', 'codmunr', 'codpre', 'codsub', 
        'dirres', 'evehistorico', 'fecaju', 'fecarcxl', 'fmfuerza', 'fmgrado', 'fmunidad', 'fecdef', 'fechanto', 'numide', 
        'priape', 'prinom', 'segape', 'segnom', 'codeve', 'gpcarcela', 'gpdesmovi', 'gpdesplaz', 'gpdiscapa', 'gpgestan', 
        'gpindigen', 'gpmadcom', 'gpmigrant', 'gpotros', 'gppobicbf', 'gppsiquia', 'gpvicvio', 'localidad', 'ndepnotif', 
        'ndepresi', 'nitupgd', 'nmunnotif', 'nmunresi', 'nomdilf', 'nomupgd', 'ocupacion', 'teldilf', 'telefono', 'tipide', 
        'vereda', 'coddptor', 'codmunr', 'evehistorico', 'nomestfcaso',
        'acumliqui', 'agente', 'artralgia', 'aumhemato', 'caidaplaq', 'cefalea', 'choque', 'clasfinal', 'coddepd', 'codmund', 
        'codpaisd', 'codpaisr', 'conducta', 'control', 'dañoorgan', 'desplazami', 'diarrea', 'direclabor', 'dolorabdo', 'dolrretroo', 
        'edadconvertida', 'erupcionr', 'estrato', 'extravasac', 'famantdngu', 'fecexa', 'fecexp', 'fecrec', 'fiebre', 'fuente', 
        'hemmucosa', 'hemorrhem', 'hepatomeg', 'hipotensio', 'hipotermia', 'labajuste', 'malgias', 'muesbazo', 'muescerebr', 
        'mueshigado', 'muesmedula', 'muesmiocar', 'muespulmon', 'muesriñon', 'muestra', 'muesttejid', 'nacionali', 'ndepproce.1', 
        'ndepresi.1', 'nmunnotif.1', 'nmunproce.1', 'nmunresi.1', 'nombrenacionalidad', 'nombres', 'nomeve.1', 'nomgrupo', 'npaisproce', 
        'npaisresi', 'numide.1', 'nunimodif', 'prueba', 'resultado', 'semges', 'somnolenci', 'unimodif', 'valor', 'version', 'vomito',
        'coddptor', 'codmunr', 'evehistorico', 'consecutive', 'consecutiveorigen','departamentonotificacion','municipionotificacion',
        'nacionalidad','paisocurrencia', 'paisresidencia', 'partición', 'vasispro', 'consecutive12', 'particion', 'coddpton',
        'confirmados', 'departamentoresidencia', 'gppobicfb', 'grupob', 'municipioresidencia', 'estadofinaldecaso', 'codmunn', 
        
    ]

    for nombre, data in dataframes.items():
        cols_a_borrar = [col for col in delete if col in data.columns]
        if cols_a_borrar:
            dataframes[nombre] = data.drop(columns=cols_a_borrar)
            print(f"🗑️ En '{nombre}' se eliminaron: {cols_a_borrar}")
        else:
            print(f"✅ En '{nombre}' no había columnas para eliminar.")

    return dataframes

In [6]:
dfs_col = delete(dfs_col, "Colombia")
dfs_atl = delete(dfs_atl, "Atlantico")


check_cols(dfs_col, "Colombia")
check_cols(dfs_atl, "Atlantico")


=======Colombia======

🗑️ En 'Datos_2018_210.csv' se eliminaron: ['ajuste', 'cbmte', 'cerdef', 'codase', 'coddptor', 'codpaiso', 'codmunr', 'codpre', 'codsub', 'fecaju', 'fecarcxl', 'fmfuerza', 'fmgrado', 'fmunidad', 'fecdef', 'fechanto', 'codeve', 'gpcarcela', 'gpdesmovi', 'gpdesplaz', 'gpdiscapa', 'gpgestan', 'gpindigen', 'gpmadcom', 'gpmigrant', 'gpotros', 'gppsiquia', 'gpvicvio', 'nomupgd', 'ocupacion', 'coddptor', 'codmunr', 'nomestfcaso', 'codpaisr', 'estrato', 'fuente', 'nombrenacionalidad', 'nomgrupo', 'semges', 'coddptor', 'codmunr', 'consecutive', 'consecutiveorigen', 'departamentonotificacion', 'municipionotificacion', 'nacionalidad', 'paisocurrencia', 'paisresidencia', 'vasispro', 'particion', 'coddpton', 'confirmados', 'departamentoresidencia', 'gppobicfb', 'grupob', 'municipioresidencia', 'estadofinaldecaso', 'codmunn']
🗑️ En 'Datos_2018_220.csv' se eliminaron: ['ajuste', 'barver', 'cbmte', 'cenpobla', 'cerdef', 'codase', 'coddptor', 'codpaiso', 'codmunr', 'codpre', 'cod

🗑️ En 'Datos_2019_220.csv' se eliminaron: ['ajuste', 'barver', 'cbmte', 'cenpobla', 'cerdef', 'codase', 'coddptor', 'codpaiso', 'codmunr', 'codpre', 'codsub', 'fecaju', 'fmfuerza', 'fmgrado', 'fmunidad', 'fecdef', 'fechanto', 'codeve', 'gpcarcela', 'gpdesmovi', 'gpdesplaz', 'gpdiscapa', 'gpgestan', 'gpindigen', 'gpmadcom', 'gpmigrant', 'gpotros', 'gppsiquia', 'gpvicvio', 'localidad', 'ocupacion', 'vereda', 'coddptor', 'codmunr', 'nomestfcaso', 'estrato', 'fuente', 'nombrenacionalidad', 'nomgrupo', 'semges', 'coddptor', 'codmunr', 'consecutive', 'departamentonotificacion', 'municipionotificacion', 'nacionalidad', 'coddpton', 'confirmados', 'departamentoresidencia', 'gppobicfb', 'grupob', 'municipioresidencia', 'estadofinaldecaso', 'codmunn']
🗑️ En 'Datos_2020_210.csv' se eliminaron: ['ajuste', 'cbmte', 'cerdef', 'codase', 'coddptor', 'codpaiso', 'codmunr', 'codpre', 'codsub', 'fecaju', 'fecarcxl', 'fmfuerza', 'fmgrado', 'fmunidad', 'fecdef', 'fechanto', 'codeve', 'gpcarcela', 'gpdesmovi

## Unificación

In [7]:
# Concatenar archivos en un solo DataFrame

def concatenar(dataframes, unidad):
    print(f"\n======={unidad}======\n")
    data = pd.concat(dataframes.values(), ignore_index=True)
    renombres = {
        "coddptoo": "cod_dep",
        "codmuno": "cod_mun",
        "confin":"desenlace",
        "ndepproce": "departamento",
        "nmunproce": "municipio",
        "fecnot": "fecha_notific",
        "feccon": "fecha_consulta",
        "fechos": "fecha_hospital",
        "inisin": "fecha_ini_sint",
        "nomeve": "evento",
        "pachos": "hospitalizado",
        "peretn": 'etnia',
        "tipcas": "tipo_caso",
        "tipss": "regimen_salud",
        'ano': 'año',
        'departamentoocurrencia': 'departamento',
        'municipioocurrencia': 'municipio', 
        'nombreevento': 'evento'
    }
    data.rename(columns=renombres, inplace=True)
    data = data[sorted(data.columns)]
    print(data.info())
    return data

In [8]:
data_col = concatenar(dfs_col, "Colombia")
data_atl = concatenar(dfs_atl, "Atlantico")


=======Colombia======

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 491869 entries, 0 to 491868
Data columns (total 20 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   area            491869 non-null  int64 
 1   año             491869 non-null  int64 
 2   cod_dep         491869 non-null  int64 
 3   cod_mun         491869 non-null  int64 
 4   departamento    491869 non-null  object
 5   desenlace       491869 non-null  int64 
 6   edad            491869 non-null  int64 
 7   etnia           491869 non-null  int64 
 8   evento          491869 non-null  object
 9   fecha_consulta  491868 non-null  object
 10  fecha_hospital  238132 non-null  object
 11  fecha_ini_sint  491869 non-null  object
 12  fecha_notific   491869 non-null  object
 13  hospitalizado   491869 non-null  int64 
 14  municipio       491869 non-null  object
 15  regimen_salud   491869 non-null  object
 16  semana          491869 non-null  int64 
 17  sexo 

In [ ]:
mapeo_dep = {
    'ATLANTICO' : 'ATLÁNTICO', 
    'BOLIVAR': 'BOLÍVAR',
    'BOYACA': 'BOYACÁ',
    'CAQUETA': 'CAQUETÁ',
    'CHOCO': 'CHOCÓ',
    'CORDOBA': 'CÓRDOBA', 
    'GUAJIRA': 'LA GUAJIRA', 
    'GUAINIA': 'GUIANÍA',
    'NARIï¿½O': 'NARIÑO',
    'QUINDIO': 'QUINDÍO', 
    'SAN ANDRES': 'ARCHIPIÉLAGO DE SAN ANDRÉS, PROVIDENCIA Y SANTA CATALINA',
    'VALLE': 'VALLE DEL CAUCA', 
    'VAUPES': 'VAUPÉS'
}
data_col['departamento'] = data_col['departamento'].apply(
    lambda x: mapeo_dep.get(x, x)
)

In [9]:
data_col.head()

,area,año,cod_dep,cod_mun,departamento,desenlace,edad,etnia,evento,fecha_consulta,fecha_hospital,fecha_ini_sint,fecha_notific,hospitalizado,municipio,regimen_salud,semana,sexo,tipo_caso,unimed
0,1,2018,44,430,GUAJIRA,1,14,6,DENGUE,2018-02-22,NaN,2018-02-16,2018-02-22,2,MAICAO,S,7,F,3,1
1,1,2018,44,430,GUAJIRA,1,16,6,DENGUE,2018-01-18,2018-01-19,2018-01-14,2018-01-19,1,MAICAO,S,3,M,3,1
2,1,2018,50,590,META,1,15,6,DENGUE,2018-11-20,NaN,2018-11-15,2018-11-20,2,PUERTO RICO,S,46,F,2,1
3,1,2018,50,1,META,1,15,6,DENGUE,2018-06-02,NaN,2018-06-01,2018-06-03,2,VILLAVICENCIO,S,22,M,2,1
4,2,2018,50,313,META,1,16,6,DENGUE,2018-09-30,NaN,2018-09-28,2018-09-30,2,GRANADA,S,39,M,2,1


In [11]:
data_atl.head()

,area,año,cod_dep,cod_mun,departamento,desenlace,edad,etnia,evento,fecha_consulta,fecha_hospital,fecha_ini_sint,fecha_notific,hospitalizado,municipio,regimen_salud,semana,sexo,tipo_caso,unimed
0,1,2018,8,849,ATLANTICO,1,6,6,DENGUE,06/11/2018,- -,04/11/2018,09/11/2018,2,USIACURI,C,45,F,2,1
1,1,2018,8,1,ATLANTICO,1,24,6,DENGUE,23/02/2018,23/02/2018,16/02/2018,01/03/2018,1,BARRANQUILLA,C,7,F,3,1
2,1,2018,8,758,ATLANTICO,1,26,6,DENGUE,10/01/2018,- -,07/01/2018,10/01/2018,2,SOLEDAD,C,2,F,3,1
3,1,2018,8,758,ATLANTICO,1,29,6,DENGUE,28/09/2018,02/10/2018,28/09/2018,02/10/2018,1,SOLEDAD,C,39,F,3,1
4,1,2018,8,1,ATLANTICO,1,29,6,DENGUE,10/01/2018,- -,09/01/2018,18/01/2018,2,BARRANQUILLA,C,2,F,3,1


_________

## Agregación

In [12]:
# [Variable] Ciclo vital a partir de edad en años
def ciclo_vital(edad):
    if edad == 0:
        return "Desconocido"
    elif edad <= 5:             # 1-5
        return "Infancia menor"
    elif edad <= 11:            # 6-11
        return "Infancia mayor"
    elif edad <= 17:            # 12-17
        return "Adolescencia"
    elif edad <=59:             # 18-59
        return "Adulto"
    else:                       # 60+
        return "Adulto mayor"
    
# [Variable] Periodo epidemiológico (4 semanas)
def asignar_periodo(semana):
    """
    Calcula el número de periodo según la semana epidemiológica.
    Cada periodo abarca 4 semanas. 
    La semana 53 se incluye en el último periodo.
    """
    if pd.isna(semana):
        return np.nan
    semana = int(semana)
    periodo = int(np.ceil(semana / 4))
    if semana == 53:  # Semana extra del año
        periodo = int(np.ceil(52 / 4))  # igual al último periodo
    return periodo


# [Mapeo] Variables categóricas
mapeo = {
    'area' : {1: 'Cabecera municipal' , 2:'Centro poblado', 3:'Rural disperso'}, 
    'hospitalizado': {1: 'Si', 2: 'No'},
    'desenlace': {1: 'Vivo', 2: 'Fallecido'},
    'etnia': {1: 'Indigena', 2: 'Rom', 3: 'Raizal', 4: 'Palenquero', 5: 'Negro', 6:'Otro'},
    'regimen_salud': {'P': 'Excepción', 'E': 'Especial', 'C': 'Contributivo', 'S': 'Subsidiado', 'N': 'No asegurado', 'I': 'Indeterminado'},
    'sexo': {'M': 'Masculino', 'F': 'Femenino'},
    'tipo_caso': {2: 'Probable', 3: 'Conf. por laboratorio', 5: 'Conf. por nexo'}
}
# [Mapeo] Grupos étnicos
mapeo_etnico = {
    'Indigena': 'Indígena',
    'Negro': 'Afrocolombiano',
    'Raizal': 'Afrocolombiano',
    'Palenquero': 'Afrocolombiano',
    'Rom': 'Rom',
    'Otro': 'Otro'
}


def agreg(data, unidad):
    print(f"\n======={unidad}======\n")

    # [Variable] Evento
    data.loc[:, 'evento'] = data['evento'].str.strip()
    data['evento'] = np.where(data['evento'] == 'DENGUE', 'Clasico', 
                        np.where(data['evento'] == 'DENGUE GRAVE', 'Grave', 'Otros'))
    print(data['evento'].unique())

    # [Variable] Edad en años
    data = data[data['edad'] != 0]
    data["edad_años"] = np.where(data["unimed"] == 1, data["edad"],   # ya está en años
                        np.where(data["unimed"] == 2, data["edad"] / 12,  # meses → años
                        np.where(data["unimed"] == 3, data["edad"] / 365, # días → años
                        np.where(data["unimed"] == 4, data["edad"] / (365*24), # horas
                        np.where(data["unimed"] == 5, data["edad"] / (365*24*60), # minutos
                        np.where(data["unimed"] == 0, np.nan, 
                        np.nan))))))  # por si hay otros valores
    data["edad_años"] = data["edad_años"].round().fillna(-1).astype(int)

    # [Variable] Ciclo vital
    data["ciclo_vital"] = data["edad_años"].apply(ciclo_vital)
    print(data["ciclo_vital"].unique())
    
    # [Variable] Mapeo de caracteres 
    data = data.replace(mapeo)
    data['grupo_etnico'] = data['etnia'].map(mapeo_etnico)
    
    # [Variable] Periodo
    data["periodo"] = data["semana"].apply(asignar_periodo)

    # Descartar variables originales ya transformadas
    data = data.drop(columns=['edad', 'unimed', 'etnia', 'cod_dep', 'cod_mun'])
    
    print(data.shape)
    print(data.columns)
    return data

In [13]:
data_col = agreg(data_col, 'Colombia')
data_atl = agreg(data_atl, 'Atlantico')


=======Colombia======

['Clasico' 'Grave']
['Adolescencia' 'Adulto' 'Infancia mayor' 'Adulto mayor' 'Infancia menor'
 'Desconocido']
(491869, 19)
Index(['area', 'año', 'departamento', 'desenlace', 'evento', 'fecha_consulta',
       'fecha_hospital', 'fecha_ini_sint', 'fecha_notific', 'hospitalizado',
       'municipio', 'regimen_salud', 'semana', 'sexo', 'tipo_caso',
       'edad_años', 'ciclo_vital', 'grupo_etnico', 'periodo'],
      dtype='object')

=======Atlantico======

['Clasico' 'Grave']
['Infancia mayor' 'Adulto' 'Infancia menor' 'Adolescencia' 'Adulto mayor'
 'Desconocido']
(31284, 19)
Index(['area', 'año', 'departamento', 'desenlace', 'evento', 'fecha_consulta',
       'fecha_hospital', 'fecha_ini_sint', 'fecha_notific', 'hospitalizado',
       'municipio', 'regimen_salud', 'semana', 'sexo', 'tipo_caso',
       'edad_años', 'ciclo_vital', 'grupo_etnico', 'periodo'],
      dtype='object')


In [14]:
def obtener_subregion(mun):
    if mun in ['BARRANQUILLA', 'GALAPA', 'MALAMBO', 'SOLEDAD', 'PUERTO COLOMBIA']:
        return 'Metropolitana'
    elif mun in ['PALMAR DE VARELA', 'PONEDERA', 'SANTO TOMÁS', 'SABANAGRANDE']:
        return 'Oriental'
    elif mun in ['CAMPO DE LA CRUZ', 'CANDELARIA', 'SANTA LUCÍA', 'SUAN', 'REPELÓN', 'MANATÍ']:
        return 'Sur'
    elif mun in ['BARANOA', 'POLONUEVO', 'LURUACO', 'USIACURÍ', 'SABANALARGA']:
        return 'Centro'
    elif mun in ['JUAN DE ACOSTA', 'PIOJÓ', 'TUBARÁ']:
        return 'Costera'
    else:
        return 'Desconocida'
    

def atl(data):


    # [Variable] Departamento
    data = data.copy()
    data['departamento'] = (
        data["departamento"]
        .astype(str)
        .str.upper()
        .str.strip()
        .str.replace('_', ' ', regex=False)
        .replace({'ATLANTICO': 'ATLÁNTICO'})
    )
    data = data[data['departamento'] == "ATLÁNTICO"]
    print(data['departamento'].unique())

    # [Variable] Departamento

    muni = {
        'REPELON': 'REPELÓN',
        'USIACURI': 'USIACURÍ',
        'REPELON': 'REPELÓN', 
        'SANTO TOMAS': 'SANTO TOMÁS', 
        'TUBARA': 'TUBARÁ',
        '* ATLANTICO. MUNICIPIO DESCONO': 'DESCONOCIDO',
        'MANATI': 'MANATÍ',
        'POLO NUEVO': 'POLONUEVO',
        'PIOJO': 'PIOJÓ',
        'SANTA LUCIA': 'SANTA LUCÍA'
    }

    data['municipio'] = (
        data['municipio']
        .astype(str)
        .str.upper()
        .str.strip()
        .str.replace('_', ' ', regex=False)
    )
    data = data.replace(muni)

    data = data[data['municipio'] != "DESCONOCIDO"]

    # Aplicamos la función
    data['subregion'] = data['municipio'].apply(obtener_subregion)

    # Reducir a Barranquilla
    data = data[data['municipio'] != "BARRANQUILLA"]

    print(data['municipio'].unique())
    
    return data

data_atl = atl(data_atl)

['ATLÁNTICO']
['USIACURÍ' 'SOLEDAD' 'POLONUEVO' 'GALAPA' 'BARANOA' 'MALAMBO' 'PONEDERA'
 'SABANALARGA' 'JUAN DE ACOSTA' 'PUERTO COLOMBIA' 'TUBARÁ'
 'PALMAR DE VARELA' 'LURUACO' 'SABANAGRANDE' 'MANATÍ' 'SUAN' 'PIOJÓ'
 'SANTO TOMÁS' 'CANDELARIA' 'REPELÓN' 'CAMPO DE LA CRUZ' 'SANTA LUCÍA']


In [15]:
# [Data] Estructuración por temática

    # Variables demográficas
demograficas = [
    "sexo",                    # M/F
    "edad_años",               # Edad numérica
    "ciclo_vital",             # Etapa de vida
    "grupo_etnico",            # Grupo étnico declarado
    "regimen_salud"            # Régimen (contributivo, subsidiado, etc.)
]

    # Variables del evento epidemiológico
evento = [
    "evento",                  # Clásico, grave
    "tipo_caso",               # Confirmado, probable, descartado
    "hospitalizado",            # Sí/No
    "desenlace"                # Recuperado, fallecido, etc.
]

    # Variables de temporalidad
temporalidad = [
    "año",                      # Año del evento
    "semana",                   # Semana epidemiológica
    "periodo",                  # Periodo (cada 4 semanas)
    "fecha_ini_sint",           # Inicio de síntomas
    "fecha_consulta",           # Fecha de consulta
    "fecha_hospital",           # Fecha de hospitalización
    "fecha_notific",            # Fecha de notificación
]

    # Variables espaciales
espacial = [
    "departamento",
    "municipio",                # Municipio de ocurrencia
    "subregion",                # Subregion del Atlántico
    "area",                     # Urbana o rural
]

orden_columnas = demograficas + evento + temporalidad + espacial

def org(data): 
    cols_existentes = [col for col in orden_columnas if col in data.columns]
    data = data[cols_existentes + [col for col in data.columns if col not in cols_existentes]]
    print(data.info())
    return data

data_col = org(data_col)
data_atl = org(data_atl)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 491869 entries, 0 to 491868
Data columns (total 19 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   sexo            491869 non-null  object
 1   edad_años       491869 non-null  int64 
 2   ciclo_vital     491869 non-null  object
 3   grupo_etnico    491869 non-null  object
 4   regimen_salud   491869 non-null  object
 5   evento          491869 non-null  object
 6   tipo_caso       491869 non-null  object
 7   hospitalizado   491869 non-null  object
 8   desenlace       491869 non-null  object
 9   año             491869 non-null  int64 
 10  semana          491869 non-null  int64 
 11  periodo         491869 non-null  int64 
 12  fecha_ini_sint  491869 non-null  object
 13  fecha_consulta  491868 non-null  object
 14  fecha_hospital  238132 non-null  object
 15  fecha_notific   491869 non-null  object
 16  departamento    491869 non-null  object
 17  municipio       491869 non-nu

In [16]:
data_col.to_csv("../DATA/registros_colombia.csv", index=False, encoding='latin-1')
data_atl.to_csv("../DATA/registros_atlantico.csv", index=False, encoding='latin-1')

_____________________

In [17]:
# Total por municipio y año
total_atl_año = data_atl.groupby(["municipio", "año"]).size().reset_index(name="casos_por_año_atl")

# Pivotear años como columnas
tabla_atl_año = total_atl_año.pivot(index="municipio", columns="año", values="casos_por_año_atl").fillna(0).astype(int)
tabla_atl_año["Total"] = tabla_atl_año.sum(axis=1)
tabla_atl_año.reset_index(inplace=True)

display(tabla_atl_año)

tabla_atl_año.to_csv("../DATA/casos_x_año.csv", index=False)

año,municipio,2018,2019,2020,2021,2022,2023,Total
0,BARANOA,162,395,170,109,96,89,1021
1,CAMPO DE LA CRUZ,0,30,6,15,79,17,147
2,CANDELARIA,1,28,2,1,13,7,52
3,GALAPA,136,62,35,166,362,76,837
4,JUAN DE ACOSTA,49,34,9,27,99,31,249
5,LURUACO,5,49,31,168,12,6,271
6,MALAMBO,157,164,221,730,245,158,1675
7,MANATÍ,4,49,6,48,137,9,253
8,PALMAR DE VARELA,6,23,16,8,16,31,100
9,PIOJÓ,7,0,6,10,10,36,69
